In [3]:
import json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime as dt 
from elasticsearch import Elasticsearch
import tweepy
from config import *

es = Elasticsearch()


class TweetStreamListener(StreamListener):

    def on_data(self, data):

        # decode json
        dict_data = json.loads(data)
       
    
    
    
        coord = dict_data["coordinates"]
        if coord is not None:
            coord = dict_data["coordinates"]
            lan = dict_data["coordinates"][0]
            lat = dict_data["coordinates"][1]
        else:
            coord = "None"
    
    
        tweet_date = dict_data['created_at']
        username = dict_data['user']['name']
        account_creation_date = dict_data['created_at']
        user_description = dict_data['user']['description']
        user_url = dict_data['user']['url']
        verified_status = dict_data['user']['verified']
        source = dict_data['source'][55:]
        friends_count = dict_data['user']['friends_count']
        followers_count = dict_data['user']['followers_count']
        retweeted_count = dict_data['retweet_count']
        favorite_count = dict_data['favorite_count']
        hashtags = dict_data['entities']['hashtags']
        location = dict_data["user"]["location"]
        
        tweet_full_text = dict_data['text']
        
        # preparando os dados para o elasticsearch
        doc = {
            '@timestamp': dt.now(),
            'data_tweet': tweet_date,
            'nome': str(username),
            'perfil_criado': str(account_creation_date),
            'descricao_usuario': str(user_description),
            'url_usuario': str(user_url),
            'user_verificado': bool(verified_status),
            'source': str(source),
            'seguindo': int(friends_count),
            'seguidores': int(followers_count),
            'retweets': int(retweeted_count),
            'likes': int(favorite_count),
            'hashtags': hashtags,
            'location': str(location),
            'tweet_full_text': str(tweet_full_text),
            'word_list': str(tweet_full_text).split(' '),
            "coordinates": coord
        }
        
        
        
        # add text and sentiment info to elasticsearch
        es.index(index="twitter_indexx", body=doc)
#                  
                 
#                  body={dict_data["user"]["screen_name"],
#                           dict_data["timestamp_ms"],
#                           dict_data["created_at"], # unfortunately this gets stored as a string
#                           dict_data["user"]["location"], # user location
#                           dict_data["text"])
        
        for d in doc.items():
            print(d)
            
        print('')              
        return True

    # on failure
    def on_error(self, status):
        print(status)

if __name__ == '__main__':

    # create instance of the tweepy tweet stream listener
    listener = TweetStreamListener()

    # set twitter keys/tokens
    consumer_key="YbT3Qc3mPzSHuSVUOag3JYGHj"
    consumer_secret="GMWhjULPjJ3P0OUs6qvUEOe6ZndU7gTKHhIp8EM2dZJrrJjGfU"
    access_token="1396467357724315650-eJIyojDd7tMdBtNVKYhCS10Wu9yV3b"
    access_token_secret="ehBPo6sUyoD2G0u5QO9uQ5vBPkUBJ5EGti0ayJYDSuwhq"
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    # create instance of the tweepy stream
    stream = Stream(auth, listener)
    
    stream.sample(languages=['pt'])

    # search twitter for "congress" keyword
    stream.filter(track=["messi", "psg", "barcelona"])
    
   

('@timestamp', datetime.datetime(2021, 8, 9, 10, 54, 53, 863568))
('data_tweet', 'Mon Aug 09 13:54:48 +0000 2021')
('nome', 'Mari.')
('perfil_criado', 'Mon Aug 09 13:54:48 +0000 2021')
('descricao_usuario', 'Guimarães')
('url_usuario', 'None')
('user_verificado', False)
('source', 'llow">Twitter for Android</a>')
('seguindo', 256)
('seguidores', 119)
('retweets', 0)
('likes', 0)
('hashtags', [])
('location', 'None')
('tweet_full_text', 'RT @cfaria_16: quase nada do que é vivido é postado, mas vcs continuam a achar que sabem de tudo')
('word_list', ['RT', '@cfaria_16:', 'quase', 'nada', 'do', 'que', 'é', 'vivido', 'é', 'postado,', 'mas', 'vcs', 'continuam', 'a', 'achar', 'que', 'sabem', 'de', 'tudo'])
('coordinates', 'None')

('@timestamp', datetime.datetime(2021, 8, 9, 10, 54, 53, 882543))
('data_tweet', 'Mon Aug 09 13:54:48 +0000 2021')
('nome', 'nanaabyssϟ')
('perfil_criado', 'Mon Aug 09 13:54:48 +0000 2021')
('descricao_usuario', '1992, qualquer pronome. 1994, fan account.')
('url_u

KeyboardInterrupt: 